In [24]:
import warnings
warnings.filterwarnings('ignore')

import os
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import time

from itertools import product
from tqdm.notebook import tqdm
from statsmodels.stats.diagnostic import acorr_ljungbox
import pandas as pd
import numpy as np
import os
import ast

from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import (
    mean_absolute_percentage_error,
    mean_absolute_error,
    mean_squared_error,
    r2_score
)

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, SimpleRNN, LSTM
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow as tf
from tensorflow.keras.regularizers import l2

# REGRESIÓN

## Lectura de datos

In [25]:
df = pd.read_csv('./datasets/data_treino_dv_df_2000_2010.csv')
df.head(1)

,HORA (UTC),"VENTO, DIREï¿½ï¿½O HORARIA (gr) (ï¿½ (gr))","VENTO, VELOCIDADE HORARIA (m/s)",UMIDADE REL. MAX. NA HORA ANT. (AUT) (%),UMIDADE REL. MIN. NA HORA ANT. (AUT) (%),TEMPERATURA Mï¿½XIMA NA HORA ANT. (AUT) (ï¿½C),TEMPERATURA Mï¿½NIMA NA HORA ANT. (AUT) (ï¿½C),"UMIDADE RELATIVA DO AR, HORARIA (%)","PRESSAO ATMOSFERICA AO NIVEL DA ESTACAO, HORARIA (mB)","PRECIPITAï¿½ï¿½O TOTAL, HORï¿½RIO (mm)","VENTO, RAJADA MAXIMA (m/s)",PRESSï¿½O ATMOSFERICA MAX.NA HORA ANT. (AUT) (mB),PRESSï¿½O ATMOSFERICA MIN. NA HORA ANT. (AUT) (mB)
0,12:00,0.809017,1.8,69.0,60.0,22.6,20.7,61.0,888.2,0.0,3.8,888.2,887.7


In [26]:
df.columns = ['HORA','WIND_DIR_HOR','WIND_VEL_HOR','HUM_REL_MAX_ANT','HUM_REL_MIN_ANT','TEMP_MAX_ANT','TEMP_MIN_ANT','HUM_REL_HOR','PRES_ATM_NIV','PREC_HOR','RAFAGA_VIENTO','PRES_ATM_MAX_ANT','PRES_ATM_MIN_ANT']
df.head(2)

,HORA,WIND_DIR_HOR,WIND_VEL_HOR,HUM_REL_MAX_ANT,HUM_REL_MIN_ANT,TEMP_MAX_ANT,TEMP_MIN_ANT,HUM_REL_HOR,PRES_ATM_NIV,PREC_HOR,RAFAGA_VIENTO,PRES_ATM_MAX_ANT,PRES_ATM_MIN_ANT
0,12:00,0.809017,1.8,69.0,60.0,22.6,20.7,61.0,888.2,0.0,3.8,888.2,887.7
1,13:00,0.965926,2.7,62.0,55.0,24.2,22.5,55.0,888.4,0.0,4.7,888.4,888.2


In [27]:
df.drop(columns='HORA', inplace= True)

## Redes Neuronales

In [28]:
# Listar dispositivos disponibles
devices = tf.config.list_physical_devices('GPU')

if devices:
    print(f"TensorFlow está utilizando la GPU: {devices}")
else:
    print("TensorFlow no está utilizando la GPU")

TensorFlow está utilizando la GPU: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


### LSTM

In [ ]:
#df = df.iloc[:24*21*5]
df = df.iloc[-24*21*5:]
df.shape

In [ ]:
# 🧠 Genera secuencias de entrenamiento para RNN y LSTM
def create_sequences(df, target_col, window_size):
    X, y = [], []
    for i in range(len(df) - window_size):
        seq = df.iloc[i:i+window_size]
        X.append(seq.drop(columns=[target_col]).values)
        y.append(df.iloc[i+window_size][target_col])
    return np.array(X), np.array(y)

# 🧠 Construye un modelo LSTM para dependencias temporales largas
def build_lstm_model(timesteps, input_dim, hidden_units, activation='tanh', learning_rate=0.001):
    model = Sequential()
    model.add(LSTM(hidden_units, activation=activation, input_shape=(timesteps, input_dim)))
    model.add(Dense(1))
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss='mse')
    return model

In [ ]:
def sliding_window_lstm_only(
    df,
    target_col='WIND_VEL_HOR',
    T_values=[7, 14,21],
    test_window=1,
    lstm_params={
        'hidden_units': [32, 64],
        'activation': ['tanh'],
        'learning_rate': [0.001],
        'epochs': [50],
        'batch_size': [32]
    },
    save_path='./progreso'
):
    os.makedirs(save_path, exist_ok=True)
    resultados_por_T = {}

    log_tiempos_path = os.path.join(save_path, 'tiempos_ejecucion.csv')
    if not os.path.exists(log_tiempos_path):
        with open(log_tiempos_path, 'w') as f:
            f.write('T_dias,modelo,parametros,duracion_segundos\n')

    for T in tqdm(T_values, desc="Procesando ventanas T"):
        print(f"\n🧭 Iniciando evaluación para ventana T={T} días...")
        inicio_ventana = time.time()
        T_hours = T * 24
        test_hours = test_window * 24
        extra = 1 * 24  # Datos adicionales necesarios para generar las secuencias del test
        total_windows = len(df) - T_hours - test_hours - extra + 1
        output_path = os.path.join(save_path, f'LSTM_resultados_T{T}.csv')

        if os.path.exists(output_path):
            df_prev = pd.read_csv(output_path)
            if 'params' in df_prev:
                df_prev['params'] = df_prev['params'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
        else:
            df_prev = pd.DataFrame(columns=['modelo', 'params', 'T_dias', 'T_horas', 'MAPE', 'MAE', 'RMSE', 'MSE', 'R2', 'LjungBox_p'])

        param_grid = list(product(*lstm_params.values()))
        param_keys = list(lstm_params.keys())
        model_type = 'LSTM'

        for combo in param_grid:
            param_dict = dict(zip(param_keys, combo))
            ya_realizado = not df_prev.empty and ((df_prev['modelo'] == model_type) & (df_prev['params'].apply(lambda p: p == param_dict))).any()
            if ya_realizado:
                rmse = df_prev[((df_prev['modelo'] == model_type) & (df_prev['params'].apply(lambda p: p == param_dict)))]['RMSE'].values[0]
                print(f"⏩ Saltando modelo ya evaluado: {model_type} con hiperparámetros: {param_dict} y RMSE {rmse}")
                continue

            modelo_idx = param_grid.index(combo) + 1
            total_modelos = len(param_grid)
            progreso_modelo = (modelo_idx / total_modelos) * 100
            print(f"\n🔧 Modelo {modelo_idx}/{total_modelos} ({progreso_modelo:.1f}%) - {model_type} con hiperparámetros: {param_dict}")

            inicio_modelo = time.time()
            resultados = {k: [] for k in ['MAPE', 'MAE', 'RMSE', 'MSE', 'R2', 'LjungBox_p']}

            for start in tqdm(range(0, total_windows, 24), desc=f"     ↪ Subventanas ({model_type})", leave=False):
                # 🪟 Segmento de datos: entrenamiento + prueba + extra para completar secuencia
                data_window = df.iloc[start: start + T_hours + test_hours + extra].copy()
                train_data = data_window.iloc[:T_hours]
                test_data = data_window.iloc[T_hours:]

                # ⚙️ Escalado
                scaler = StandardScaler()
                X_train_scaled = scaler.fit_transform(train_data.drop(columns=[target_col]))
                X_test_scaled = scaler.fit_transform(test_data.drop(columns=[target_col]))  # ⚠️ intencionalmente fit_transform en ambos

                train_scaled = pd.DataFrame(X_train_scaled, columns=train_data.columns.drop(target_col))
                train_scaled[target_col] = train_data[target_col].values

                test_scaled = pd.DataFrame(X_test_scaled, columns=test_data.columns.drop(target_col))
                test_scaled[target_col] = test_data[target_col].values

                # 📊 Unión de train y test escalados para crear secuencias
                scaled = pd.concat([train_scaled, test_scaled])

                # 🧠 Creación de secuencias tipo LSTM
                X, y = create_sequences(scaled, target_col, T_hours)

                # 🏁 División explícita del test: últimos 24 pasos (1 día) como predicción
                X_train, y_train = X[:-24], y[:-24]
                X_test, y_test = X[-24:], y[-24:]

                # 🧱 Modelo LSTM
                model = build_lstm_model(
                    timesteps=X_train.shape[1],
                    input_dim=X_train.shape[2],
                    hidden_units=param_dict['hidden_units'],
                    activation=param_dict['activation'],
                    learning_rate=param_dict['learning_rate']
                )

                early_stop = EarlyStopping(monitor='loss', patience=10, restore_best_weights=True)
                model.fit(X_train, y_train, epochs=param_dict['epochs'], batch_size=param_dict['batch_size'], verbose=0, callbacks=[early_stop])
                y_pred = model.predict(X_test).flatten()

                # 📈 Métricas
                residuals = y_test - y_pred
                resultados['MAPE'].append(mean_absolute_percentage_error(y_test, y_pred))
                resultados['MAE'].append(mean_absolute_error(y_test, y_pred))
                resultados['RMSE'].append(np.sqrt(mean_squared_error(y_test, y_pred)))
                resultados['MSE'].append(mean_squared_error(y_test, y_pred))
                resultados['R2'].append(r2_score(y_test, y_pred))
                ljung_p = acorr_ljungbox(residuals, lags=[1], return_df=True)['lb_pvalue'].iloc[0] if len(residuals) >= 2 else np.nan
                resultados['LjungBox_p'].append(ljung_p)

            # 🗃️ Guardar resultados
            nuevo_row = pd.DataFrame([{
                'modelo': model_type,
                'params': param_dict,
                'T_dias': T,
                'T_horas': T_hours,
                'MAPE': np.mean(resultados['MAPE']),
                'MAE': np.mean(resultados['MAE']),
                'RMSE': np.mean(resultados['RMSE']),
                'MSE': np.mean(resultados['MSE']),
                'R2': np.mean(resultados['R2']),
                'LjungBox_p': np.nanmean(resultados['LjungBox_p'])
            }])
            df_prev = pd.concat([df_prev, nuevo_row], ignore_index=True)
            df_prev.to_csv(output_path, index=False)

            # ⏱️ Duración por modelo
            duracion = time.time() - inicio_modelo
            with open(log_tiempos_path, 'a') as f:
                f.write(f'{T},{model_type},"{param_dict}",{duracion:.2f}\n')

            print(f"✅ Finalizado {model_type} con RMSE promedio: {np.mean(resultados['RMSE']):.4f} en {duracion:.2f} segundos")

        resultados_por_T[T] = df_prev
        duracion_ventana = time.time() - inicio_ventana
        print(f"🕒 Tiempo total para T={T} días: {duracion_ventana:.2f} segundos")

    return resultados_por_T


## Resultados

In [ ]:

def plot_rmse_bar(df, title='RMSE por Modelo', sort_ascending=True, figsize=(12, 6)):
    """
    Genera una gráfica de barras del RMSE por modelo, ordenada según el valor del RMSE.

    Parámetros:
    - df: DataFrame que debe contener las columnas 'modelo', 'params' y 'RMSE'.
    - title: Título de la gráfica.
    - sort_ascending: Booleano, si True ordena de menor a mayor RMSE.
    - figsize: Tamaño de la figura (tupla).

    Retorna:
    - None (muestra la gráfica).
    """
    df_sorted = df.sort_values(by="RMSE", ascending=sort_ascending)
    etiquetas = df_sorted['modelo'].astype(str) + ' ' + df_sorted['params'].astype(str)

    plt.figure(figsize=figsize)
    plt.bar(etiquetas, df_sorted['RMSE'])
    plt.xticks(rotation=45, ha='right')
    plt.ylabel('RMSE')
    plt.title(title)
    plt.tight_layout()
    plt.show()

Leer resultados de los modelos evaluados.

In [ ]:
## Sin escalar
df_ns_7 = pd.read_csv('./progreso/reg_resultados_T7.csv')
df_ns_14 = pd.read_csv('./progreso/reg_resultados_T14.csv')
df_ns_21 = pd.read_csv('./progreso/reg_resultados_T21.csv')

##Escalados
df_s_7 = pd.read_csv('./progreso/s_resultados_T7.csv')
df_s_14 = pd.read_csv('./progreso/s_resultados_T14.csv')
df_s_21 = pd.read_csv('./progreso/s_resultados_T21.csv')

## RNN
df_mlp_7 = pd.read_csv('./progreso/mlp_resultados_T7.csv')
df_mlp_14 = pd.read_csv('./progreso/mlp_resultados_T14.csv')
df_mlp_21 = pd.read_csv('./progreso/mlp_resultados_T21.csv')


df_w_7 = pd.concat([df_ns_7, df_s_7, df_mlp_7])
df_w_14 = pd.concat([df_ns_14, df_s_14, df_mlp_14])
df_w_21 = pd.concat([df_ns_21, df_s_21,df_mlp_21])


In [ ]:
df_mlp_7.sort_values('RMSE', ascending=True).head(1)

In [ ]:
df_w_7.sort_values('RMSE', ascending=True).head(50)

In [ ]:
# Ver los DataFrames en memoria
plot_rmse_bar(df_w_7, title='RMSE por Modelo - T=7 días (ordenado de menor a mayor)', sort_ascending=True)
